In [1]:
print(f"API Key: {api_key}")  # 调试用，确保密钥加载正常

NameError: name 'api_key' is not defined

In [23]:
import requests

url = "https://pro-api.coingecko.com/api/v3/coins/categories/list"

headers = {"accept": "application/json",
            'X-CG-PRO-API-KEY': api_key}

response = requests.get(url, headers=headers)


# 确保请求成功
if response.status_code == 200:
    categories = response.json()

    # 查找包含 'bnb' 的类别
    bnb_categories = [
        category for category in categories if 'bnb' in category['category_id'].lower() or 'bnb' in category['name'].lower()
    ]
    
    # 打印找到的类别
    if bnb_categories:
        print("Categories containing 'bnb':")
        for category in bnb_categories:
            print(f"Category ID: {category['category_id']}, Name: {category['name']}")
    else:
        print("No categories found containing 'bnb'.")
else:
    print(f"Error fetching categories: {response.status_code} - {response.text}")


Categories containing 'bnb':
Category ID: binance-smart-chain, Name: BNB Chain Ecosystem
Category ID: boba-bnb-ecosystem, Name: Boba BNB Ecosystem
Category ID: bridged-wbnb, Name: Bridged WBNB
Category ID: opbnb-ecosystem, Name: opBNB Ecosystem


In [29]:
import requests
import json
from dotenv import load_dotenv
import os

# 加载 .env 文件中的环境变量
load_dotenv()

# 获取 Binance Launchpool 的 Token
url = "https://pro-api.coingecko.com/api/v3/coins/markets"
params = {
    'vs_currency': 'usd',
    'category': 'binance-smart-chain',
    'order': 'market_cap_desc',
    'per_page': 100,
    'page': 1,
}
# 从环境变量中获取 API 密钥
api_key = os.getenv('COINGECKO_API_KEY')

# 添加 API 密钥
headers = {
    'X-CG-PRO-API-KEY': api_key
}

response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    tokens = response.json()
    token_ids = [token['id'] for token in tokens]  # 获取 Token ID 列表
    print("Token IDs:", token_ids)
else:
    print(f"Error fetching tokens: {response.status_code} - {response.text}")

Token IDs: ['binancecoin', 'chainlink', 'the-open-network', 'mantra-dao', 'uniswap', 'murasaki', 'pepe', 'aave', 'first-digital-usd', 'cosmos', 'fetch-ai', 'lombard-staked-btc', 'binance-peg-weth', 'injective-protocol', 'solv-btc', 'bonk', 'solv-protocol-solvbtc-bbn', 'dexe', 'floki', 'renzo-restaked-eth', 'bittorrent', 'wbnb', 'chain-2', 'pancakeswap-token', 'usdd', 'binance-peg-dogecoin', 'axie-infinity', 'frax', 'usdx-money-usdx', 'pendle', 'pumpbtc', 'matic-network', 'beam-2', 'insurance-2', 'compound-governance-token', 'true-usd', 'aioz-network', 'apenft', 'chex-token', 'axelar', 'trust-wallet-token', '1inch', 'frax-ether', 'safepal', 'layerzero', 'ai-companions', 'binance-peg-busd', 'oasis-network', 'zilliqa', 'staked-frax-ether', 'baby-doge-coin', 'metars-genesis', 'quorium', 'woo-network', 'mask-network', 'ankr', 'zetachain', 'aelf', 'wrapped-beacon-eth', 'cheems-token', 'swftcoin', 'stepn', 'gmt-token', 'sushi', 'project-galaxy', 'world-mobile-token', 'lorenzo-stbtc', 'solo-co

In [37]:
import requests
import json
from dotenv import load_dotenv
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm  # 导入 tqdm
import time  # 导入 time 库

# 加载 .env 文件中的环境变量
load_dotenv()

# 获取 Binance Launchpool 的 Token
url = "https://pro-api.coingecko.com/api/v3/coins/markets"
params = {
    'vs_currency': 'usd',
    'category': 'binance-smart-chain',
    'order': 'market_cap_desc',
    'per_page': 250,
    'page': 1,
}
# 从环境变量中获取 API 密钥
api_key = os.getenv('COINGECKO_API_KEY')

# 添加 API 密钥
headers = {
    'X-CG-PRO-API-KEY': api_key
}

all_token_ids = []  # 存储所有的 Token ID

# 最大重试次数
max_retries = 10
max_threads = 5  # 最大并行请求数

# 请求函数
def fetch_page(page_num):
    params['page'] = page_num
    try:
        response = requests.get(url, params=params, headers=headers)
        
        # 如果请求成功
        if response.status_code == 200:
            tokens = response.json()

            # 如果没有数据返回，说明已经获取完了所有页面
            if not tokens:
                return None

            # 获取当前页面的 Token ID 列表
            token_ids = [token['id'] for token in tokens]
            return token_ids
        else:
            print(f"Error fetching page {page_num}: {response.status_code} - {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        #print(f"Request failed on page {page_num}: {e}")
        return None

# 使用 ThreadPoolExecutor 并行请求多个页面
with ThreadPoolExecutor(max_workers=max_threads) as executor:
    futures = []
    page = 1
    while True:
        futures.append(executor.submit(fetch_page, page))  # 提交请求任务
        page += 1
        time.sleep(1)  # 添加小延时避免请求过于频繁
        
        # 如果没有返回数据，退出循环
        if len(futures) >= max_threads and len(futures) == page:
            break

    # 进度条
    for future in tqdm(as_completed(futures), desc="Fetching pages", unit="page"):
        result = future.result()
        if result:
            all_token_ids.extend(result)  # 将当前页面的 ID 添加到总列表中
        else:
            break  # 如果没有数据，结束循环

# 打印所有的 Token IDs
print("All Token IDs:", all_token_ids)

KeyboardInterrupt: 

In [33]:
len(all_token_ids)

2250

In [ ]:
import os
import requests
import json

# 添加 API 密钥
headers = {
    'X-CG-PRO-API-KEY': api_key

# 创建文件夹
category = 'binance-launchpool'
os.makedirs(category, exist_ok=True)

# 获取 Binance Launchpool 的 Token
url = "https://pro-api.coingecko.com/api/v3/coins/markets"
params = {
    'vs_currency': 'usd',
    'category': 'binance-launchpool',
    'order': 'market_cap_desc',
    'per_page': 100,
    'page': 1,
}

response = requests.get(url, params=params, headers=headers)

if response.status_code == 200:
    tokens = response.json()
    token_ids = [token['id'] for token in tokens]  # 获取 Token ID 列表
    print("Token IDs:", token_ids)

    # 第二步：获取每个 Token 的市场数据并保存为 JSON 文件
    for ticker in token_ids:
        market_chart_url = f"https://pro-api.coingecko.com/api/v3/coins/{ticker}/market_chart"
        market_chart_params = {
            'vs_currency': 'usd',
            'days': 'max',  # 获取所有可用的数据
        }

        market_chart_response = requests.get(market_chart_url, params=market_chart_params, headers=headers)
        if market_chart_response.status_code == 200:
            market_chart_data = market_chart_response.json()  # 获取市场数据
            # 创建文件名并写入 JSON 文件
            output_file = os.path.join(category, f"{ticker}_market_chart.json")
            with open(output_file, 'w') as json_file:
                json.dump(market_chart_data, json_file, indent=4)
            print(f"Data saved to {output_file}")
        else:
            print(f"Error fetching market chart for {ticker}: {market_chart_response.status_code}")
else:
    print(f"Error fetching tokens: {response.status_code} - {response.text}")

                         Token ID                     Token Name  Symbol  \
0                 atletico-madrid      Atletico Madrid Fan Token     atm   
1                            dodo                           DODO    dodo   
2                      highstreet                     Highstreet    high   
3                        litentry                       Litentry     lit   
4                    sleepless-ai                   Sleepless AI      ai   
5                 anchor-protocol                Anchor Protocol     anc   
6                           renzo                          Renzo     rez   
7                    merit-circle                   Merit Circle      mc   
8                    cyberconnect                          CYBER   cyber   
9                    wing-finance                   Wing Finance    wing   
10                          mobox                          Mobox    mbox   
11             unifi-protocol-dao             Unifi Protocol DAO    unfi   
12          

In [1]:
import os
import requests
import json
import pandas as pd

def fetch_and_analyze_market_data(category):
    # 创建文件夹
    os.makedirs(category, exist_ok=True)

    # 添加 API 密钥
    headers = {
        'X-CG-PRO-API-KEY': api_key
    }

    # 获取 Token
    url = "https://pro-api.coingecko.com/api/v3/coins/markets"
    params = {
        'vs_currency': 'usd',
        'category': category,
        'order': 'market_cap_desc',
        'per_page': 100,
        'page': 1,
    }

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        tokens = response.json()
        # 获取 Token ID、名称和符号
        token_info = {token['id']: {'name': token['name'], 'symbol': token['symbol']} for token in tokens}

        # 获取每个 Token 的市场数据并保存为 JSON 文件
        for ticker in token_info.keys():
            market_chart_url = f"https://pro-api.coingecko.com/api/v3/coins/{ticker}/market_chart"
            market_chart_params = {
                'vs_currency': 'usd',
                'days': 'max',  # 获取所有可用的数据
            }

            market_chart_response = requests.get(market_chart_url, params=market_chart_params, headers=headers)
            if market_chart_response.status_code == 200:
                market_chart_data = market_chart_response.json()  # 获取市场数据
                # 创建文件名并写入 JSON 文件
                output_file = os.path.join(category, f"{ticker}_market_chart.json")
                with open(output_file, 'w') as json_file:
                    json.dump(market_chart_data, json_file, indent=4)
            else:
                print(f"Error fetching market chart for {ticker}: {market_chart_response.status_code}")

        # 分析市场数据
        results = []
        for filename in os.listdir(category):
            if filename.endswith('_market_chart.json'):
                file_path = os.path.join(category, filename)
                
                # 读取 JSON 文件
                with open(file_path, 'r') as json_file:
                    data = json.load(json_file)
                
                # 提取价格和交易量数据
                prices = data.get('prices', [])
                volumes = data.get('total_volumes', [])
                
                # 确保有足够的数据进行分析
                if len(prices) >= 31:
                    first_day_price = prices[0][1]  # 1st day closing price
                    second_day_price = prices[1][1]  # 2nd day closing price
                    thirty_first_day_price = prices[30][1]  # 31st day closing price

                    price_change_1_day = (second_day_price / first_day_price - 1) * 100  # 1日变化百分比
                    price_change_30_days = (thirty_first_day_price / first_day_price - 1) * 100  # 30日变化百分比

                    volume_1_day = volumes[0][1]  # 1st day volume
                    volume_31_day = volumes[30][1]  # 31st day volume

                    # 计算交易量变化百分比
                    volume_change_1_day = (volumes[1][1] / volume_1_day - 1) * 100 if volume_1_day else 0
                    volume_change_30_days = (volume_31_day / volume_1_day - 1) * 100 if volume_1_day else 0

                    # 获取 Token 名称、符号和上市日期
                    token_name = filename.split('_')[0]  # 从文件名中提取 Token ID
                    listed_date = pd.to_datetime(prices[0][0], unit='ms').date()  # 获取上市日期
                    token_details = token_info.get(token_name, {"name": "N/A", "symbol": "N/A"})  # 获取 Token 详情

                    results.append({
                        'Token ID': token_name,
                        'Token Name': token_details['name'],  # Token 名称
                        'Symbol': token_details['symbol'],  # Token 符号
                        'Listed Date': listed_date,  # 上市日期
                        '1st Day Price': first_day_price,
                        '2nd Day Price': second_day_price,
                        'Price Change 1 Day (%)': price_change_1_day,
                        '31st Day Price': thirty_first_day_price,
                        'Price Change 30 Days (%)': price_change_30_days,
                        '1st Day Volume': volume_1_day,
                        '2nd Day Volume': volumes[1][1],
                        '31st Day Volume': volume_31_day,
                        'Volume Change 1 Day (%)': volume_change_1_day,
                        'Volume Change 30 Days (%)': volume_change_30_days,
                    })

        # 转换为 DataFrame
        df = pd.DataFrame(results)

        # 保存为 CSV 文件
        csv_file_path = os.path.join(category, f"{category}_market_analysis.csv")
        df.to_csv(csv_file_path, index=False)
        print(f"Data saved to {csv_file_path}")

        return df

    else:
        print(f"Error fetching tokens: {response.status_code} - {response.text}")
        return None

# 使用示例
category_name = 'binance-launchpool'
df_result = fetch_and_analyze_market_data(category_name)

NameError: name 'api_key' is not defined

In [ ]:
category_name = 'binance-launchpool'
df_result = fetch_and_analyze_market_data(category_name)

/Users/andyouyang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Data saved to binance_pengu_ohlcv_data.csv
Data saved to okx_pengu_ohlcv_data.csv
Data saved to bybit_pengu_ohlcv_data.csv
